# Interest Rate Products Construction Tutorial

This notebook demonstrates how to construct various interest rate products using the contract system. We use QuantLib under the hood for proper business calendars, day count conventions, and schedule generation.

Products covered:
1. Fixed Rate Leg
2. Floating Rate Leg
3. Interest Rate Swap
4. European Swaptions (Physical & Cash)
5. Bermudan Cancellable Swap

In [9]:
import inspect
from datetime import datetime
from rates import fixed_leg, float_leg, swap, physical_swaption, cash_settled_swaption, bermudan_cancellable
from contract import Observation, Contract, Leg

## 1. Fixed Rate Leg

Let's create a 5-year fixed rate leg with semi-annual payments.

In [14]:
# Create 5Y fixed rate leg
fixed = fixed_leg(
    effective_date="2025-02-28",
    termination_date="2030-02-28",
    fixed_rate=0.03,  # 3.00%
    notional=10_000_000,
    currency="USD",
    payment_frequency="6M",  # Semi-annual
    calendar_name="USA",
    day_count="30/360"
)

print("Fixed Rate Leg Structure:")
print(fixed)
print(inspect.getdoc(fixed_leg))
#print(inspect.getsource(fixed_leg))

Fixed Rate Leg Structure:
0.015	10000000	USD	2025-08-28
0.014916666666666667	10000000	USD	2026-02-27
0.015083333333333332	10000000	USD	2026-08-28
0.014833333333333334	10000000	USD	2027-02-26
0.01533333333333333	10000000	USD	2027-08-30
0.014833333333333334	10000000	USD	2028-02-28
0.015	10000000	USD	2028-08-28
0.015	10000000	USD	2029-02-28
0.015	10000000	USD	2029-08-28
0.015	10000000	USD	2030-02-28
Calculate fixed leg payment dates and cash flows.

Args:
    effective_date: Start date as datetime or string 'YYYY-MM-DD'
    termination_date: End date as datetime or string 'YYYY-MM-DD'
    fixed_rate: Annual fixed rate as decimal (e.g., 0.03 for 3%)
    notional: Notional amount
    currency: Currency code (e.g., 'USD', 'EUR')
    payment_frequency: String frequency specifier ('1M', '3M', '6M', '1Y', etc.)
    calendar_name: Calendar name ('USA', 'UK', 'TARGET', etc.)
    business_day_convention: Date convention ('MF'=Modified Following, 'F'=Following, 'P'=Preceding)
    date_generation: S

## 2. Floating Rate Leg

Now let's create a matching floating rate leg with quarterly payments.

In [3]:
# Create 5Y floating rate leg
floating = float_leg(
    effective_date="2025-02-28",
    termination_date="2030-02-28",
    notional=10_000_000,
    currency="USD",
    index_id="USD-LIBOR-3M",
    payment_frequency="3M",  # Quarterly
    calendar_name="USA",
    day_count="ACT/360"
)

print("Floating Rate Leg Structure:")
print(floating)

Floating Rate Leg Structure:
(USD-LIBOR-3M(2025-02-26) * 0.24722222222222223)	10000000	USD	2025-05-28
(USD-LIBOR-3M(2025-05-23) * 0.25555555555555554)	10000000	USD	2025-08-28
(USD-LIBOR-3M(2025-08-26) * 0.25555555555555554)	10000000	USD	2025-11-28
(USD-LIBOR-3M(2025-11-25) * 0.25277777777777777)	10000000	USD	2026-02-27
(USD-LIBOR-3M(2026-02-25) * 0.25)	10000000	USD	2026-05-28
(USD-LIBOR-3M(2026-05-26) * 0.25555555555555554)	10000000	USD	2026-08-28
(USD-LIBOR-3M(2026-08-26) * 0.2611111111111111)	10000000	USD	2026-11-30
(USD-LIBOR-3M(2026-11-25) * 0.24444444444444444)	10000000	USD	2027-02-26
(USD-LIBOR-3M(2027-02-24) * 0.25277777777777777)	10000000	USD	2027-05-28
(USD-LIBOR-3M(2027-05-26) * 0.2611111111111111)	10000000	USD	2027-08-30
(USD-LIBOR-3M(2027-08-26) * 0.25277777777777777)	10000000	USD	2027-11-29
(USD-LIBOR-3M(2027-11-24) * 0.25277777777777777)	10000000	USD	2028-02-28
(USD-LIBOR-3M(2028-02-24) * 0.25555555555555554)	10000000	USD	2028-05-30
(USD-LIBOR-3M(2028-05-25) * 0.25)	10000

## 3. Interest Rate Swap

Combine fixed and floating legs into a swap.

In [4]:
# Create 5Y swap
swap_contract = swap(
    effective_date="2025-02-28",
    termination_date="2030-02-28",
    notional=10_000_000,
    fixed_rate=0.03,
    currency="USD",
    index_id="USD-LIBOR-3M",
    fixed_payment_frequency="6M",
    float_payment_frequency="3M"
)

print("Swap Structure:")
print(swap_contract)

Swap Structure:
0.015	10000000	USD	2025-08-28
0.014916666666666667	10000000	USD	2026-02-27
0.015083333333333332	10000000	USD	2026-08-28
0.014833333333333334	10000000	USD	2027-02-26
0.01533333333333333	10000000	USD	2027-08-30
0.014833333333333334	10000000	USD	2028-02-28
0.015	10000000	USD	2028-08-28
0.015	10000000	USD	2029-02-28
0.015	10000000	USD	2029-08-28
0.015	10000000	USD	2030-02-28
(USD-LIBOR-3M(2025-02-26) * 0.24722222222222223)	-10000000	USD	2025-05-28
(USD-LIBOR-3M(2025-05-23) * 0.25555555555555554)	-10000000	USD	2025-08-28
(USD-LIBOR-3M(2025-08-26) * 0.25555555555555554)	-10000000	USD	2025-11-28
(USD-LIBOR-3M(2025-11-25) * 0.25277777777777777)	-10000000	USD	2026-02-27
(USD-LIBOR-3M(2026-02-25) * 0.25)	-10000000	USD	2026-05-28
(USD-LIBOR-3M(2026-05-26) * 0.25555555555555554)	-10000000	USD	2026-08-28
(USD-LIBOR-3M(2026-08-26) * 0.2611111111111111)	-10000000	USD	2026-11-30
(USD-LIBOR-3M(2026-11-25) * 0.24444444444444444)	-10000000	USD	2027-02-26
(USD-LIBOR-3M(2027-02-24) * 0.2527

## 4. Swaptions

### 4.1 Physical Settlement

In [5]:
# Create physically settled swaption
phys_swaption = physical_swaption(
    option_expiry=datetime(2025, 2, 28),
    underlying_swap=swap_contract,
    counterparty="BANK_A"
)

print("Physical Swaption Structure:")
print(phys_swaption)

Physical Swaption Structure:
if Call(2025-02-28) is positive then
    0.015	10000000	USD	2025-08-28
    0.014916666666666667	10000000	USD	2026-02-27
    0.015083333333333332	10000000	USD	2026-08-28
    0.014833333333333334	10000000	USD	2027-02-26
    0.01533333333333333	10000000	USD	2027-08-30
    0.014833333333333334	10000000	USD	2028-02-28
    0.015	10000000	USD	2028-08-28
    0.015	10000000	USD	2029-02-28
    0.015	10000000	USD	2029-08-28
    0.015	10000000	USD	2030-02-28
    (USD-LIBOR-3M(2025-02-26) * 0.24722222222222223)	-10000000	USD	2025-05-28
    (USD-LIBOR-3M(2025-05-23) * 0.25555555555555554)	-10000000	USD	2025-08-28
    (USD-LIBOR-3M(2025-08-26) * 0.25555555555555554)	-10000000	USD	2025-11-28
    (USD-LIBOR-3M(2025-11-25) * 0.25277777777777777)	-10000000	USD	2026-02-27
    (USD-LIBOR-3M(2026-02-25) * 0.25)	-10000000	USD	2026-05-28
    (USD-LIBOR-3M(2026-05-26) * 0.25555555555555554)	-10000000	USD	2026-08-28
    (USD-LIBOR-3M(2026-08-26) * 0.2611111111111111)	-10000000	USD	2

### 4.2 Cash Settlement

In [6]:
# Create cash-settled swaption
cash_swaption = cash_settled_swaption(
    option_expiry=datetime(2026, 2, 28),
    swap_rate_ticker="USD-SWAP-5Y",  # 5Y swap rate observation
    notional=10_000_000,
    currency="USD",
    strike=0.03,
    payment_date=datetime(2026, 3, 2),  # T+2 settlement
    option_type="Call"
)

print("Cash-Settled Swaption Structure:")
print(cash_swaption)

Cash-Settled Swaption Structure:
max((USD-SWAP-5Y(2026-02-28) - 0.03), 0))	10000000	USD	2026-03-02


## 5. Bermudan Cancellable Swap

Create a Bermudan cancellable swap with quarterly call dates. Note that call dates are independent of payment dates, following market convention.

In [7]:
# Call dates aligned with market convention
call_dates = [
    datetime(2026, 2, 28),  # First call date
    datetime(2027, 2, 28),
    datetime(2028, 2, 28),
    datetime(2029, 2, 28)
]  # Call dates are independent of payment dates

# Create Bermudan cancellable structure
bermudan = bermudan_cancellable(
    exercise_dates=call_dates,
    underlying_swap=swap_contract,
    counterparty="BANK_A",
    option_type="Call"
)

print("Bermudan Cancellable Swap Structure:")
print(bermudan)

Bermudan Cancellable Swap Structure:
0.015	10000000	USD	2025-08-28
0.014916666666666667	10000000	USD	2026-02-27
(USD-LIBOR-3M(2025-02-26) * 0.24722222222222223)	-10000000	USD	2025-05-28
(USD-LIBOR-3M(2025-05-23) * 0.25555555555555554)	-10000000	USD	2025-08-28
(USD-LIBOR-3M(2025-08-26) * 0.25555555555555554)	-10000000	USD	2025-11-28
(USD-LIBOR-3M(2025-11-25) * 0.25277777777777777)	-10000000	USD	2026-02-27
if Call(2026-02-28) is positive then
    None
else
    0.015083333333333332	10000000	USD	2026-08-28
    0.014833333333333334	10000000	USD	2027-02-26
    (USD-LIBOR-3M(2026-02-25) * 0.25)	-10000000	USD	2026-05-28
    (USD-LIBOR-3M(2026-05-26) * 0.25555555555555554)	-10000000	USD	2026-08-28
    (USD-LIBOR-3M(2026-08-26) * 0.2611111111111111)	-10000000	USD	2026-11-30
    (USD-LIBOR-3M(2026-11-25) * 0.24444444444444444)	-10000000	USD	2027-02-26
    if Call(2027-02-28) is positive then
        None
    else
        0.01533333333333333	10000000	USD	2027-08-30
        0.014833333333333334	100